In [2]:
# Numpy
import numpy as np

# Torch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

# Torchvision
import torchvision
import torchvision.transforms as transforms

# Matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# OS
import os
import argparse

%env CUDA_VISIBLE_DEVICES=1

# Set random seed for reproducibility
SEED = 87
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)


def print_model(encoder, decoder):
    print("============== Encoder ==============")
    print(encoder)
    print("============== Decoder ==============")
    print(decoder)
    print("")


def create_model():
    autoencoder = Autoencoder()
    print_model(autoencoder.encoder, autoencoder.decoder)
    if torch.cuda.is_available():
        autoencoder = autoencoder.cuda()
        print("Model moved to GPU in order to speed up training.")
    return autoencoder


def get_torch_vars(x):
    if torch.cuda.is_available():
        x = x.cuda()
    return Variable(x)

def imshow(img):
    npimg = img.cpu().numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Input size: [batch, 3, 32, 32]
        # Output size: [batch, 3, 32, 32]
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 12, 4, stride=2, padding=1),            # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.Conv2d(12, 24, 4, stride=2, padding=1),           # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.Conv2d(24, 48, 4, stride=2, padding=1),           # [batch, 48, 4, 4]
            nn.ReLU(),
# 			nn.Conv2d(48, 96, 4, stride=2, padding=1),           # [batch, 96, 2, 2]
#             nn.ReLU(),
        )
        self.decoder = nn.Sequential(
#             nn.ConvTranspose2d(96, 48, 4, stride=2, padding=1),  # [batch, 48, 4, 4]
#             nn.ReLU(),
			nn.ConvTranspose2d(48, 24, 4, stride=2, padding=1),  # [batch, 24, 8, 8]
            nn.ReLU(),
			nn.ConvTranspose2d(24, 12, 4, stride=2, padding=1),  # [batch, 12, 16, 16]
            nn.ReLU(),
            nn.ConvTranspose2d(12, 3, 4, stride=2, padding=1),   # [batch, 3, 32, 32]
            nn.Sigmoid(),
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded



env: CUDA_VISIBLE_DEVICES=1


In [3]:
device  = "cuda:0"

In [4]:

parser = argparse.ArgumentParser(description='Parameter Processing')
parser.add_argument('--dataset', type=str, default='CIFAR10', help='dataset')
parser.add_argument('--model', type=str, default='ConvNet', help='model')
parser.add_argument('--ipc', type=int, default=50, help='image(s) per class')
parser.add_argument('--eval_mode', type=str, default='SS', help='eval_mode') # S: the same to training model, M: multi architectures,  W: net width, D: net depth, A: activation function, P: pooling layer, N: normalization layer,
parser.add_argument('--num_exp', type=int, default=1, help='the number of experiments')
parser.add_argument('--num_eval', type=int, default=1, help='the number of evaluating randomly initialized models')
parser.add_argument('--epoch_eval_train', type=int, default=1000, help='epochs to train a model with synthetic data') # it can be small for speeding up with little performance drop
parser.add_argument('--Iteration', type=int, default=2000, help='training iterations')
parser.add_argument('--lr_img', type=float, default=1.0, help='learning rate for updating synthetic images')
parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
parser.add_argument('--batch_real', type=int, default=256, help='batch size for real data')
parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
parser.add_argument('--init', type=str, default='real', help='noise/real: initialize synthetic images from random noise or randomly sampled real images.')
parser.add_argument('--dsa_strategy', type=str, default='color_crop_cutout_flip_scale_rotate', help='differentiable Siamese augmentation strategy')
parser.add_argument('--data_path', type=str, default='/home/ssd7T/ZTL_gcond/data_cv', help='dataset path')
parser.add_argument('--save_path', type=str, default='result/gen', help='path to save results')
parser.add_argument('--dis_metric', type=str, default='ours', help='distance metric')
from utils import get_loops, get_dataset, get_network, get_eval_pool, evaluate_synset, get_daparam, match_loss, get_time, TensorDataset, epoch, DiffAugment, ParamDiffAug
import warnings
args = parser.parse_args([])
args.device = device
channel, im_size, num_classes, class_names, mean, std, dst_train, dst_test, testloader = get_dataset(args.dataset, args.data_path)

images_all = [torch.unsqueeze(dst_train[i][0], dim=0) for i in range(len(dst_train))]
labels_all = [dst_train[i][1] for i in range(len(dst_train))]
indices_class = [[] for c in range(num_classes)]
for i, lab in enumerate(labels_all):
    indices_class[lab].append(i)
images_all = torch.cat(images_all, dim=0).to(device)
labels_all = torch.tensor(labels_all, dtype=torch.long, device=device)

accs = []
model_eval_pool = get_eval_pool(args.eval_mode, args.model, args.model)
import copy
accs_all_exps = dict() # record performances of all experiments
for key in model_eval_pool:
    accs_all_exps[key] = []
args.dsa_param = ParamDiffAug()
args.dsa = False if args.dsa_strategy in ['none', 'None'] else True
model_eval= model_eval_pool[0]


Files already downloaded and verified
Files already downloaded and verified


In [5]:
img_syn = []
label_syn = []
img_real_train = []
label_real_train = []

for i in range(50):
    try:
    
        img_syn_ = torch.load(f'img_syn_{i}.pt')
        label_syn_ = torch.load(f'label_syn_{i}.pt')
        # pairs_real_=torch.load(f'pairs_real_{i}.pt')
        indexs_real_=torch.load(f'indexs_real_{i}.pt')
        
        # img_real_train_ = torch.cat(pairs_real_, dim=0)
        
    
        label_real_ = []
        for c in range(num_classes):
            idx_shuffle = indexs_real_[c]
            label_real_.append(labels_all[idx_shuffle].to("cpu"))
        # img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
        label_real_train_ = torch.from_numpy(np.concatenate(label_real_, axis=0))
        # label_real_train_ = torch.cat(label_real_train_, dim=0)
        
        img_syn.append(img_syn_)
        label_syn.append(label_syn_)
        # img_real_train.append(img_real_train_)
        label_real_train.append( label_real_train_)
        if i == 3 or i == 22 or i == 42 or i == 62:
            pairs_real_=torch.load(f'pairs_real_{i}.pt')
            img_real_train_ = torch.cat(pairs_real_, dim=0)
            img_real_train.append(img_real_train_)
            
    except:
        pass


img_syn = torch.cat(img_syn, dim=0).to(device)
label_syn = torch.cat(label_syn, dim=0).to(device)
img_real_train = torch.cat(img_real_train, dim=0).to(device)
label_real_train = torch.cat(label_real_train, dim=0).to(device)



In [6]:
img_syn_test = []
label_syn_test = []
img_real_train_test = []
label_real_train_test = []

for i in range(10):
    try:
        i +=60
        img_syn_ = torch.load(f'img_syn_{i}.pt')
        label_syn_ = torch.load(f'label_syn_{i}.pt')
        # pairs_real_=torch.load(f'pairs_real_{i}.pt')
        indexs_real_=torch.load(f'indexs_real_{i}.pt')
        
        # img_real_train_ = torch.cat(pairs_real_, dim=0)
        
    
        label_real_ = []
        for c in range(num_classes):
            idx_shuffle = indexs_real_[c]
            label_real_.append(labels_all[idx_shuffle].to("cpu"))
        # img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
        label_real_train_ = torch.from_numpy(np.concatenate(label_real_, axis=0))
        # label_real_train_ = torch.cat(label_real_train_, dim=0)
        
        img_syn_test.append(img_syn_)
        label_syn_test.append(label_syn_)
        # img_real_train.append(img_real_train_)
        label_real_train_test.append( label_real_train_)
        if i == 3 or i == 22 or i == 42 or i == 62:
            pairs_real_=torch.load(f'pairs_real_{i}.pt')
            img_real_train_ = torch.cat(pairs_real_, dim=0)
            img_real_train_test.append(img_real_train_)
            
    except:
        pass


img_syn_test = torch.cat(img_syn_test, dim=0).to(device)
label_syn_test = torch.cat(label_syn_test, dim=0).to(device)
img_real_train_test = torch.cat(img_real_train_test, dim=0).to(device)
label_real_train_test = torch.cat(label_real_train_test, dim=0).to(device)


In [40]:
img_syn = []
label_syn = []
img_real_train = []
label_real_train = []

for i in range(64):
    try:
    
        img_syn_ = torch.load(f'img_syn_{i}.pt')
        label_syn_ = torch.load(f'label_syn_{i}.pt')
        # pairs_real_=torch.load(f'pairs_real_{i}.pt')
        indexs_real_=torch.load(f'indexs_real_{i}.pt')
        
        # img_real_train_ = torch.cat(pairs_real_, dim=0)
        
    
        label_real_ = []
        for c in range(num_classes):
            idx_shuffle = indexs_real_[c]
            label_real_.append(labels_all[idx_shuffle].to("cpu"))
        # img_real = torch.from_numpy(np.concatenate(img_real, axis=0))
        label_real_train_ = torch.from_numpy(np.concatenate(label_real_, axis=0))
        # label_real_train_ = torch.cat(label_real_train_, dim=0)
        
        img_syn.append(img_syn_)
        label_syn.append(label_syn_)
        # img_real_train.append(img_real_train_)
        label_real_train.append( label_real_train_)
        if i == 3 or i == 22 or i == 42 or i == 62:
            pairs_real_=torch.load(f'pairs_real_{i}.pt')
            img_real_train_ = torch.cat(pairs_real_, dim=0)
            img_real_train.append(img_real_train_)
            
    except:
        pass


img_syn = torch.cat(img_syn, dim=0).to(device)
label_syn = torch.cat(label_syn, dim=0).to(device)
img_real_train = torch.cat(img_real_train, dim=0).to(device)
label_real_train = torch.cat(label_real_train, dim=0).to(device)



In [43]:
label_real_train.shape


torch.Size([6500])

In [42]:
img_real_train.shape

torch.Size([6500, 3, 32, 32])

30

In [7]:
# torch.save(img_syn, 'img_syn.pt')
# torch.save(label_syn, 'label_syn.pt')
# # 读取tensor
# img_syn = torch.load('img_syn.pt')
# label_syn = torch.load('label_syn.pt')

# img_syn =img_syn.to(device)
# label_syn =label_syn.to(device)
# device = img_syn.device

# Create model
autoencoder = create_model().to(device)

# Load data
transform = transforms.Compose(
    [transforms.ToTensor(), ])

============== Encoder ==============
Sequential(
  (0): Conv2d(3, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): Conv2d(12, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): Conv2d(24, 48, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): ReLU()
)
============== Decoder ==============
Sequential(
  (0): ConvTranspose2d(48, 24, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (1): ReLU()
  (2): ConvTranspose2d(24, 12, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (3): ReLU()
  (4): ConvTranspose2d(12, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
  (5): Sigmoid()
)

Model moved to GPU in order to speed up training.


In [8]:
img_real = []
label_real = []
for c in range(num_classes):
    idx_shuffle = np.random.permutation(indices_class[c])
    img_real.append(images_all[idx_shuffle].to("cpu") )
    label_real.append(labels_all[idx_shuffle].to("cpu"))
img_real_all = torch.from_numpy(np.concatenate(img_real, axis=0))
label_real_all = torch.from_numpy(np.concatenate(label_real, axis=0))

In [9]:
img_real_test = []
label_real_test = []
for c in range(num_classes):
    idx_shuffle = np.random.permutation(indices_class[c])[:50]
    img_real_test.append(images_all[idx_shuffle].to("cpu") )
    label_real_test.append(labels_all[idx_shuffle].to("cpu"))
img_real_test = torch.from_numpy(np.concatenate(img_real_test, axis=0))
label_real_test = torch.from_numpy(np.concatenate(label_real_test, axis=0))

In [8]:
# 用全部数据集训

In [9]:


num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_real_all[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)) 
    batch_img = batch_img.to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    loss = criterion(outputs, batch_img)
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


  

        

In [10]:
torch.save(autoencoder,"autoencoder_all.pt")

In [11]:
# 恢复原图性能
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

/home/wangkai/.local/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/home/wangkai/.local/lib/python3.9/site-packages/torch/nn/functional.py:4298: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/wangkai/.local/lib/python3.9/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


[2023-10-09 14:23:37] Evaluate_01: epoch = 1000 train time = 30 s train loss = 0.003723 train acc = 1.0000, test acc = 0.3631

Evaluate 1 random ConvNet, mean = 0.3631 std = 0.0000
-------------------------


In [11]:
# 用合成数据集微调
autoencoder = torch.load('autoencoder_all.pt') 

In [13]:
# 由原图恢复原图性能
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))

data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

[2023-10-09 16:46:50] Evaluate_01: epoch = 1000 train time = 30 s train loss = 0.027596 train acc = 0.9980, test acc = 0.3593

Evaluate 3 random ConvNet, mean = 0.2755 std = 0.1174
-------------------------


In [ ]:
# 1.用全部 cifar样本训练一个auto encoder A
# 2.用 蒸馏前原图-蒸留后图片 分别作为输入和生成目标来finetune A，得到B
# 3.选五百个原图，分别输入A和B，测试B会不会效果更好

# SEED = 87 
# A - 恢复1500训练用生成图 0.2336 —— 向训练好的AE里输入训练用1500原图，用输出结果和1500原图的label训练CONVE3，获得TEST ACC
# A - 恢复1500测试用生成图 0.2755 —— 向训练好的AE里输入测试用1500原图，用输出结果和1500测试原图的label训练CONVE3，获得TEST ACC

# A + 5000张合成图FINETUN - 恢复训练用1500原图 0.3693 —— 向训练好的AE里输入训练用1500原图，用输出结果和1500原图的label训练CONVE3，获得TEST ACC
# A + 5000张合成图FINETUN - 恢复测试用1500原图 0.3794 —— 向训练好的AE里输入测试用1500原图，用输出结果和1500测试原图的label训练CONVE3，获得TEST ACC
# A + 5000张合成图FINETUN - 恢复测试用1500生成图 0.3748 —— 向训练好的AE里输入测试用1500原图，用输出结果和1500生成图的label训练CONVE3，获得TEST ACC

# A + 5000张样本对FINETUN - 恢复训练用1500原图 0.3661
# A + 5000张样本对FINETUN - 恢复测试用原图 0.3904
# A + 5000张样本对FINETUN - 恢复测试用1500生成图 0.3639



In [44]:
# 由原图恢复合成图性能

with torch.no_grad():
  output = autoencoder(img_real_train.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_syn) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

/home/wangkai/.local/lib/python3.9/site-packages/torch/nn/functional.py:4298: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/home/wangkai/.local/lib/python3.9/site-packages/torch/nn/functional.py:4236: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(


[2023-10-09 14:53:56] Evaluate_01: epoch = 1000 train time = 486 s train loss = 0.005050 train acc = 0.9989, test acc = 0.3962

Evaluate 2 random ConvNet, mean = 0.3796 std = 0.0166
-------------------------


In [ ]:
# 1.用全部 cifar样本训练一个auto encoder A
# 2.用 蒸馏前原图-蒸留后图片 分别作为输入和生成目标来finetune A，得到B
# 3.选五百个原图，分别输入A和B，测试B会不会效果更好

# SEED = 87 + test 500张原图
# A - 恢复500原图 0.3631
# A - 恢复6500生成图 0.3796

# A + 6500张合成图FINETUN - 恢复500原图 0.3766
# A + 6500张合成图FINETUN - 恢复6500生成图 0.3907

# A + 6500张样本对FINETUN - 恢复500原图 0.3853
# A + 6500张样本对FINETUN - 恢复6500生成图 0.3926

# A + 6500张合成图FINETUN(冻结encoder) - 恢复500原图 0.3627
# A + 6500张合成图FINETUN(冻结encoder) - 恢复6500生成图 0.3878

# A + 6500张样本对FINETUN(冻结encoder) - 恢复500原图 0.3785
# A + 6500张样本对FINETUN(冻结encoder) - 恢复6500生成图 0.3913


In [1]:
# 用合成数据集微调
autoencoder = torch.load('autoencoder_all.pt') 

NameError: name 'torch' is not defined

In [46]:
# A + 13*50张合成图FINETUN

num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))
    batch_img = img_syn[c*batch:(c+1)*batch].reshape((batch, 3, 32, 32)) 
    batch_img = batch_img.to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img)
    loss = criterion(outputs, batch_img)
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



In [47]:
# 由原图恢复原图性能
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))

data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

[2023-10-09 14:55:32] Evaluate_01: epoch = 1000 train time = 59 s train loss = 0.006343 train acc = 0.9980, test acc = 0.3705

Evaluate 3 random ConvNet, mean = 0.3766 std = 0.0142
-------------------------


In [48]:
# 由原图恢复合成图性能

with torch.no_grad():
  output = autoencoder(img_real_train.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_syn) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

[2023-10-09 15:08:06] Evaluate_01: epoch = 1000 train time = 750 s train loss = 0.002400 train acc = 1.0000, test acc = 0.4331

Evaluate 4 random ConvNet, mean = 0.3907 std = 0.0274
-------------------------


In [49]:
# 用合成数据集微调
autoencoder = torch.load('autoencoder_all.pt') 

In [50]:

# A + 13*50张蒸馏前原图-蒸留后图片FINETUN(B)
num_classes = 10
batch = 50
num_feat = 3072
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters())
# model = Autoencoder(num_feat).to(device)  
# 训练
for epoch in range(1000):

  total_loss = 0
  for c in range(num_classes):
                # 获取类别c的合成图像和类别中心
                # image_syn[c*args.ipc:(c+1)*args.ipc].reshape((args.ipc, channel, im_size[0], im_size[1]))

    
    # idx_shuffle = np.random.permutation(indices_class[c])[:batch]
    batch_img_real = img_real_train[c*batch:(c+1)*batch].reshape((batch, channel, im_size[0], im_size[1])).to(device) 
    
    batch_img = img_syn[c*batch:(c+1)*batch].reshape((batch, channel, im_size[0], im_size[1])).to(device) 
    
    # batch_img = batch_img_real.to(device) 
    
    # ============ Forward ============
    encoded, outputs = autoencoder(batch_img_real)
    loss = criterion(outputs, batch_img)
    # ============ Backward ============
    # print(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



In [51]:
# 由原图恢复原图性能
with torch.no_grad():
  output = autoencoder(img_real_test.to(device))

data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_real_test) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))
# [2023-10-06 20:45:24] Evaluate_00: epoch = 1000 train time = 61 s train loss = 0.000965 train acc = 1.0000, test acc = 0.5001
# Evaluate 1 random ConvNet, mean = 0.5001 std = 0.0000

[2023-10-09 15:09:43] Evaluate_01: epoch = 1000 train time = 59 s train loss = 0.019111 train acc = 0.9980, test acc = 0.3635

Evaluate 5 random ConvNet, mean = 0.3853 std = 0.0268
-------------------------


In [52]:
# 由原图恢复合成图性能

with torch.no_grad():
  output = autoencoder(img_real_train.to(device))
data_save = []
net_eval = get_network(model_eval, channel, num_classes, im_size).to(device) # get a random model
image_syn_eval, label_syn_eval = copy.deepcopy(output[1]), copy.deepcopy(label_syn) # avoid any unaware modification
_, acc_train, acc_test = evaluate_synset(1, net_eval, image_syn_eval, label_syn_eval, testloader, args)
accs.append(acc_test)
print()
print('Evaluate %d random %s, mean = %.4f std = %.4f\n-------------------------'%(len(accs), model_eval, np.mean(accs), np.std(accs)))

[2023-10-09 15:20:34] Evaluate_01: epoch = 1000 train time = 648 s train loss = 0.003775 train acc = 0.9998, test acc = 0.4293

Evaluate 6 random ConvNet, mean = 0.3926 std = 0.0295
-------------------------
